In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QFileDialog, QMessageBox, QTextEdit, QLabel, QTableWidget, QTableWidgetItem, QHBoxLayout
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt

class DataAnalysisGUI(QWidget):
    def __init__(self):
        super().__init__()

        self.init_ui()

    def init_ui(self):
        # Set window title
        self.setWindowTitle("Intrusion Detection System")

        # Set background color to black and text color to light red
        self.setStyleSheet("background-color: rgb(0, 0, 0); color: rgb(255, 0, 0);")

        # Add watermark
        watermark_label = QLabel(self)
        watermark_label.setGeometry(20, 20, 200, 50)
        watermark_label.setStyleSheet("font-size: 20px; color: rgba(255, 255, 255, 0.3);")
        watermark_label.setText("IDS")

        # Upload button
        self.upload_button = QPushButton("Upload Dataset(s)", self)
        self.upload_button.clicked.connect(self.upload_datasets)

        # Analyze button
        self.analyze_button = QPushButton("Analyze", self)
        self.analyze_button.clicked.connect(self.analyze_data)
        self.analyze_button.setEnabled(False)  # Disable analyze button initially

        # Results text edit
        self.results_textedit = QTextEdit(self)
        self.results_textedit.setReadOnly(True)

        # Table widget for displaying results
        self.results_table = QTableWidget(self)
        self.results_table.setColumnCount(4)
        self.results_table.setHorizontalHeaderLabels(['Attack', '# training packets', '# normal test packets', '# malicious test packets'])

        # Layout for results (text edit and table)
        results_layout = QVBoxLayout()
        results_layout.addWidget(self.results_textedit)
        results_layout.addWidget(self.results_table)

        # Layout for buttons
        buttons_layout = QVBoxLayout()
        buttons_layout.addWidget(self.upload_button)
        buttons_layout.addWidget(self.analyze_button)

        # Horizontal layout for buttons and results
        horizontal_layout = QHBoxLayout()
        horizontal_layout.addLayout(buttons_layout)
        horizontal_layout.addLayout(results_layout)

        # Main layout
        main_layout = QVBoxLayout()
        main_layout.addWidget(watermark_label)
        main_layout.addLayout(horizontal_layout)
        self.setLayout(main_layout)

        self.resize(800, 600)  # Increase GUI size
        self.show()

    def upload_datasets(self):
        file_paths, _ = QFileDialog.getOpenFileNames(self, "Open Dataset(s)", "", "CSV Files (*.csv);;Text Files (*.txt);;All Files (*)")
        if file_paths:
            print(f"Selected file(s): {file_paths}")
            self.file_paths = file_paths
            self.analyze_button.setEnabled(True)  # Enable analyze button after file upload

    def analyze_data(self):
        try:
            # Iterate over each selected dataset
            for file_path in self.file_paths:
                # Load dataset based on file extension
                if file_path.endswith('.csv'):
                    # Read CSV file with low_memory set to False to avoid DtypeWarning
                    df = pd.read_csv(file_path, low_memory=False)
                elif file_path.endswith('.txt'):
                    df = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated values (TSV) format
                else:
                    QMessageBox.warning(self, "Unsupported File", f"Unsupported file format for {file_path}. Skipping.")
                    continue  # Skip to the next file
                
                # Perform data analysis
                analysis_result = f"Contents of {file_path}:\n"
                analysis_result += str(df) + "\n"
                
                # Display results in QTextEdit
                self.results_textedit.append(analysis_result)
                
                # Display results in QTableWidget
                self.display_results_table(df)
                
                # Generate and display bar chart
                self.generate_bar_chart(df)
            
            # Display a message box indicating analysis completion
            QMessageBox.information(self, "Analysis Completed", "Data analysis completed successfully.")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"An error occurred during data analysis: {str(e)}")
    
    def display_results_table(self, df):
        self.results_table.setRowCount(len(df))
        for i, row in df.iterrows():
            for j, value in enumerate(row):
                self.results_table.setItem(i, j, QTableWidgetItem(str(value)))
    
    def generate_bar_chart(self, df):
        # Grouping by attack and summing up packet counts
        attack_data = df.groupby('Attack').sum().reset_index()
        
        # Plotting bar chart
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.bar(attack_data['Attack'], attack_data['# training packets'], color='blue', label='# training packets')
        ax.bar(attack_data['Attack'], attack_data['# normal test packets'], color='green', label='# normal test packets', bottom=attack_data['# training packets'])
        ax.bar(attack_data['Attack'], attack_data['# malicious test packets'], color='red', label='# malicious test packets', bottom=attack_data['# training packets'] + attack_data['# normal test packets'])
        ax.set_xlabel('Attack')
        ax.set_ylabel('Packet Count')
        ax.set_title('Packet Count by Attack')
        ax.set_xticklabels(attack_data['Attack'], rotation=45, ha='right')
        ax.legend()
        
        # Add the plot to a canvas
        canvas = FigureCanvas(fig)
        
        # Clear layout and add the canvas
        layout = self.layout()
        layout.itemAt(1).layout().addWidget(canvas)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    gui = DataAnalysisGUI()
    sys.exit(app.exec_())


Selected file(s): ['C:/Users/ASD/ids/Dataset Properties.csv']


C:\Users\ASD\AppData\Local\Temp\ipykernel_14648\890773443.py:127: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(attack_data['Attack'], rotation=45, ha='right')
